## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras import optimizers

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'], axis =1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Look at APPLICATION_TYPE value counts for binning
count_app = application_df['APPLICATION_TYPE'].value_counts()
count_app

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(count_app[count_app < 100].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
count_class = application_df['CLASSIFICATION'].value_counts()
count_class 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
count_class_1 = count_class.loc[count_class > 1]
count_class_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(count_class[count_class < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)

In [10]:
application_dummies.shape

(34299, 45)

In [11]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop(['IS_SUCCESSFUL'], axis=1)
y = application_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model2():
    nn_model = tf.keras.models.Sequential()

    # First hidden layer
    nn_model.add(tf.keras.layers.Dense(30, activation='relu', input_dim=len(X_train_scaled[0])))
    # Middle layers
    middle_layers_sizes = [10]
    for i in range(len(middle_layers_sizes)):
        nn_model.add(tf.keras.layers.Dense(units=middle_layers_sizes[i], activation='relu'))
       
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    return nn_model
   

In [14]:
# Compile the model
# Train the model

loss_list = ["binary_crossentropy", "MSE","MAE"]
for loss in loss_list:
    print(loss)
    nn_model = create_model2()
    nn_model.compile(loss=loss, optimizer = "Adam", metrics = ['accuracy'])
    nn_model.fit(X_train_scaled, y_train, epochs=10)
    model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

binary_crossentropy
Epoch 1/10
804/804 [==============================] - 12s 11ms/step - loss: 0.5806 - accuracy: 0.7154
Epoch 2/10
804/804 [==============================] - 7s 9ms/step - loss: 0.5586 - accuracy: 0.7296
Epoch 3/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5544 - accuracy: 0.7310
Epoch 4/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7315
Epoch 5/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7324
Epoch 6/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7331
Epoch 7/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7342
Epoch 8/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7334
Epoch 9/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7338
Epoch 10/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5

In [15]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Opt_loss.h5')